In [2]:
# @title Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import requests
from matplotlib import colors
from matplotlib import rcParams
from matplotlib.colors import ListedColormap
from numpy import pi
from copy import copy
import ipywidgets as widgets  # interactive display

# @title Figure settings
rcParams['figure.figsize'] = [20, 4]
rcParams['font.size'] = 11
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True


/Users/az/Learn/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
data = pd.read_csv("data01_direction4priors.csv")
data.head()

,trial_index,trial_time,response_arrow_start_angle,motion_direction,motion_coherence,estimate_x,estimate_y,reaction_time,raw_response_time,prior_std,prior_mean,subject_id,experiment_name,experiment_id,session_id,run_id
0,1,0.000000,NaN,225,0.12,-1.749685,-1.785666,NaN,NaN,10,225,1,data01_direction4priors,11,1,1
1,2,2.730730,NaN,225,0.12,-1.819693,-1.714269,NaN,NaN,10,225,1,data01_direction4priors,11,1,1
2,3,4.913950,NaN,235,0.06,-1.562674,-1.951422,NaN,NaN,10,225,1,data01_direction4priors,11,1,1
3,4,6.997296,NaN,225,0.06,-1.601388,-1.919781,NaN,NaN,10,225,1,data01_direction4priors,11,1,1
4,5,9.097130,NaN,215,0.24,-1.639461,-1.887371,NaN,NaN,10,225,1,data01_direction4priors,11,1,1


In [4]:
# Calculate estimated degree in a new field with this name
# estimated_degree = np.degrees(np.arctan2(data['estimate_x'], data['estimate']))
data['estimated_degree'] = (np.degrees(np.arctan2(data['estimate_x'], data['estimate_y'])) + 360) % 360
print(data['estimated_degree'].head())


0    224.416887
1    226.708718
2    218.687309
3    219.833224
4    220.979140
Name: estimated_degree, dtype: float64


In [ ]:
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np

# Widget for choosing subject
subject_options = sorted(data['subject_id'].unique())
@widgets.interact(subject=widgets.Dropdown(options=subject_options, description='Subject:'))
def plot_histograms(subject):
    # Filter data for selected subject
    subj_data = data[data['subject_id'] == subject]
    # Get unique prior_std and motion_coherence values
    prior_stds = sorted(subj_data['prior_std'].dropna().unique())
    motion_coherences = sorted(subj_data['motion_coherence'].dropna().unique())
    
    n_rows = len(prior_stds)
    n_cols = len(motion_coherences)
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(4*n_cols, 3*n_rows), sharex=True, sharey=True)
    if n_rows == 1 and n_cols == 1:
        axes = np.array([[axes]])
    elif n_rows == 1:
        axes = axes[np.newaxis, :]
    elif n_cols == 1:
        axes = axes[:, np.newaxis]
    
    for i, prior_std in enumerate(prior_stds):
        for j, motion_coh in enumerate(motion_coherences):
            ax = axes[i, j]
            # Filter for this prior_std and motion_coherence
            filt = (subj_data['prior_std'] == prior_std) & (subj_data['motion_coherence'] == motion_coh)
            trials = subj_data[filt]
            # Get true direction and estimated degree, drop NaN
            true_dir = trials['motion_direction'].dropna()
            est_deg = trials['estimated_degree'].dropna()
            # Plot histograms
            # print(trials.shape)
            bins = np.linspace(0, 360, 37)  # 10 degree bins
            ax.hist(true_dir % 360, bins=bins, alpha=0.6, label='True Direction', color='tab:blue')
            ax.hist(est_deg % 360, bins=bins, alpha=0.6, label='Estimated Degree', color='tab:orange')
            ax.set_title(f'Prior std: {prior_std}, Coh: {motion_coh}')
            ax.set_xlim(0, 360)
            ax.set_xticks([0, 90, 180, 270, 360])
            if i == n_rows - 1:
                ax.set_xlabel('Degrees')
            if j == 0:
                ax.set_ylabel('Count')
            ax.legend()
    plt.tight_layout()
    plt.show()


interactive(children=(Dropdown(description='Subject:', options=(np.int64(1), np.int64(2), np.int64(3), np.int6…

In [20]:
# Compute log likelihood for fitting basic Bayesian model

# Assume: 
# - data has columns: 'motion_direction' (true direction), 'estimated_degree' (subject's estimate), 
#   'prior_mean', 'prior_std', 'motion_coherence', 'subject_id'
# - We fit a model where the subject's estimate is a noisy Bayesian posterior mean of the true direction,
#   with noise (likelihood std) as a free parameter.

from scipy.stats import norm

def compute_log_likelihood(data, likelihood_std):
    """
    Compute the log likelihood of the observed estimates under a basic Bayesian model.
    Args:
        data: DataFrame with columns 'motion_direction', 'estimated_degree', 'prior_mean', 'prior_std'
        likelihood_std: assumed std of likelihood (sensory noise)
    Returns:
        log_likelihood: float
    """
    # Drop rows with missing values
    df = data.dropna(subset=['motion_direction', 'estimated_degree', 'prior_mean', 'prior_std'])
    # Compute Bayesian posterior mean and std for each trial
    prior_var = df['prior_std'] ** 2
    likelihood_var = likelihood_std ** 2
    # Posterior mean
    post_mean = (df['motion_direction'] / likelihood_var + df['prior_mean'] / prior_var) / (1/likelihood_var + 1/prior_var)
    # Posterior std
    post_std = np.sqrt(1 / (1/likelihood_var + 1/prior_var))
    # Compute log likelihood of observed estimate under posterior
    log_likelihoods = norm.logpdf(df['estimated_degree'], loc=post_mean, scale=post_std)
    total_log_likelihood = np.sum(log_likelihoods)
    return total_log_likelihood

# Example usage:
# loglike = compute_log_likelihood(data, likelihood_std=20)
# print("Log likelihood (likelihood_std=20):", loglike)


In [11]:
# Calculate mean, median, and std of reaction time for each subject, dropping NaN values
data_coherence_012 = data[data["motion_coherence"] == 0.12]

reaction_time_stats = (
    data_coherence_012[data_coherence_012['subject_id'].isin([1, 5, 6, 10])]
        .dropna(subset=['reaction_time'])
        .groupby('subject_id')['reaction_time']
        .agg(['mean', 'median', 'std'])
)
print("Reaction time statistics by subject:")
print(reaction_time_stats)

# Calculate mean, median, and std of reaction time for the other set of subjects, dropping NaN values
reaction_time_stats_others = (
    data_coherence_012[data_coherence_012['subject_id'].isin([2, 3, 4, 7, 8, 9, 11, 12])]
        .dropna(subset=['reaction_time'])
        .groupby('subject_id')['reaction_time']
        .agg(['mean', 'median', 'std'])
)
print("\nReaction time statistics by subject (subjects 2, 3, 4, 7, 8, 9, 11, 12):")
print(reaction_time_stats_others)



Reaction time statistics by subject:
                mean    median       std
subject_id                              
1           1.491732  1.438569  0.457612
5           1.123234  1.079921  0.303347
6           1.329796  1.272664  0.354293
10          1.186519  1.129499  0.339246

Reaction time statistics by subject (subjects 2, 3, 4, 7, 8, 9, 11, 12):
                mean    median       std
subject_id                              
3           1.451398  1.409753  0.392336
7           1.579214  1.512977  0.547037
8           1.308775  1.203492  0.478837
9           1.322304  1.267986  0.395029
11          1.236222  1.187572  0.407068
12          1.463959  1.437616  0.452006


In [19]:
import numpy as np
import math

x = np.array([1, 2, 3, 4, 5])
y = np.array([1.2, 3.9, 9.1, 15.8, 25.2])  # roughly y = x^2 + noise

# Model 1: y_hat = x^2 (no parameters)
y_hat = x ** 2
residuals = y - y_hat


y_hat_simple = x ** 2
residuals_simple = y - y_hat_simple
n = len(y)
sigma2 = np.sum(residuals**2) / n
sigma2_simple = np.sum(residuals_simple**2) / n

log_likelihood_simple = -0.5 * n * np.log(2 * math.pi * sigma2_simple) - (np.sum(residuals_simple**2) / (2 * sigma2_simple))
print("Log likelihood (simple model):", log_likelihood_simple)

# Model 2: y_hat = a * x^2 + b * x + c (quadratic with parameters)
# Fit quadratic model using numpy polyfit
params = np.polyfit(x, y, 2)  # returns [a, b, c]
a, b, c = params
y_hat_quad = a * x**2 + b * x + c
residuals_quad = y - y_hat_quad
sigma2_quad = np.sum(residuals_quad**2) / n

log_likelihood_quad = -0.5 * n * np.log(2 * math.pi * sigma2_quad) - (np.sum(residuals_quad**2) / (2 * sigma2_quad))
print("Log likelihood (quadratic model):", log_likelihood_quad)
print("Quadratic model parameters: a=%.3f, b=%.3f, c=%.3f" % (a, b, c))

Log likelihood (simple model): 1.844184255993981
Log likelihood (quadratic model): 3.4672541225567635
Quadratic model parameters: a=1.064, b=-0.396, c=0.520


1.844184255993981
